### Script to calculate WFH hours for tax deductions


In [1]:
import PyPDF2
import os
import pandas as pd
import time

In [2]:
directory_path = './data2'
filenames = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f)) and not f.startswith('.DS_Store')]

In [4]:
li = []
time_start = time.time()
for file in filenames:
    reader = PyPDF2.PdfReader('data2/'+str(file))
    data = reader.pages[0].extract_text().split("\n")
    
    for i,j in enumerate(data):
        if "SALARY AND ALLOWANCES" in j:
            start = i
        if "TOTAL SALARY AND  ALLOWANCES" in j:
            end = i

    for i in reader.pages[0].extract_text().split("\n")[start+2:end]:
        li.append((i.split()[1],int(i.split()[2]), float(i.split()[-4]), float(i.split()[-2])))
time.time() - time_start

0.5167438983917236

In [5]:
df = pd.DataFrame(li, columns=['date', 'code', 'hours', 'amount'])

In [6]:
df['code'].unique()
code_desc = {2009: 'MonashConnect', 2088: 'ORAA', 2093:'RA', 2091:'Marking', 2269: 'WorkshopSupport', 2271:'AppliedClass'}

In [7]:
df['code'].unique()

array([2088, 2093, 2091, 2269, 2271, 2009, 2094])

In [8]:
df['description'] = df['code'].map(code_desc)

In [9]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m').dt.strftime('%d/%m/2022')

In [10]:
df['week']=pd.to_datetime(df['date'], format='%d/%m/%Y').dt.isocalendar().week

In [11]:
df

,date,code,hours,amount,description,week
0,09/08/2022,2088,1.00,49.42,ORAA,32
1,10/08/2022,2088,4.00,197.68,ORAA,32
2,11/08/2022,2088,6.00,296.52,ORAA,32
3,12/08/2022,2088,5.00,247.10,ORAA,32
4,16/08/2022,2088,1.00,49.42,ORAA,33
...,...,...,...,...,...,...
249,09/03/2022,2093,3.00,146.79,RA,10
250,10/03/2022,2093,2.00,97.86,RA,10
251,11/03/2022,2093,3.00,146.79,RA,10
252,17/03/2022,2093,3.00,146.79,RA,11


In [12]:
temp_df = df[df['description'].isin(['ORAA','Marking','WorkshopSupport','AppliedClass'])].groupby(['week'])['hours'].sum().reset_index()

In [13]:
temp_df

,week,hours
0,11,13.00
1,12,15.00
2,13,15.00
3,14,26.50
4,15,12.33
5,16,4.50
6,17,13.00
7,18,13.75
8,19,14.00
9,20,23.00
